In [ ]:
# The Julia boostrap block
# This should be run for the first time to install Julia kernel, and then refresh this page (e.g., Ctrl-R)
# so that colab will redirect to the installed Julia kernel
# and then doing your own work

# 1. install latest Julia using jill.py
#    tip: one can install specific Julia version using e.g., `jill install 1.7`
!pip install jill && jill install --upstream Official --confirm
# 2. install IJulia kernel
! julia -e 'using Pkg; pkg"add IJulia"; using IJulia; installkernel("Julia")'
# 3. hot-fix patch to strip the version suffix of the installed kernel so that this notebook kernelspec is version agnostic
!jupyter kernelspec install $(jupyter kernelspec list | grep julia | tr -s ' ' | cut -d' ' -f3) --replace --name julia

     |████████████████████████████████| 40 kB 2.3 MB/s 
     |████████████████████████████████| 87 kB 3.2 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=2ff0daa736f2cdce5d6c66364c74d20636f748a6e305e681f92a9737d26ff9c6
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
  Created wheel for requests-futures: filename=requests_futures-1.0.0-py3-none-any.whl size=7029 sha256=beaa5533aa06f2d1bcac94ceb2f0c3eb2ac46b86c78ad3e928bc9d4e2eb50cd8
  Stored in directory: /root/.cache/pip/wheels/e6/5d/70/8aa32975ceceb612d545b74897e6e8dcc399d78522458c3004
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=2b762871baaad51650bfb65225d5eceba379d5762b98c4dbcabf64244e2ab3c6
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built fire requests-futures wget
JILL - Julia Installer 4 Linux (MacOS, Windows and FreeBSD

In [ ]:
using Pkg
Pkg.add("CUDA")
Pkg.add("Flux")
#Pkg.add("Images")
Pkg.add("Plots")
Pkg.add("CSV")
Pkg.add("DataFrames")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed LLVMExtra_jll ─────── v0.0.10+0
   Installed OpenSpecFun_jll ───── v0.5.5+0
   Installed TimerOutputs ──────── v0.5.12
   Installed Adapt ─────────────── v3.3.1
   Installed ExprTools ─────────── v0.1.6
   Installed Requires ──────────── v1.1.3
   Installed DocStringExtensions ─ v0.8.5
   Installed Compat ────────────── v3.35.0
   Installed Random123 ─────────── v1.4.2
   Installed LogExpFunctions ───── v0.3.2
   Installed RandomNumbers ─────── v1.5.3
   Installed LLVM ──────────────── v4.5.0
   Installed ChainRulesCore ────── v1.3.1
   Installed CEnum ─────────────── v0.4.1
   Installed SpecialFunctions ──── v1.6.1
   Installed CUDA ──────────────── v3.4.2
   Installed BFloat16s ─────────── v0.1.0
   Installed GPUCompiler ───────── v0.12.9
   Installed IrrationalConstants ─ v0.1.0
   Installed Reexport ──────────── v1.2.2
   Installed AbstractFFTs ──────── v1.0.1
   Installed GPUArrays

In [ ]:
using CSV
using DataFrames
using Plots
using Flux: onehot,onehotbatch,onecold
using Flux: crossentropy,@epochs, Statistics.mean
using Flux
using Base.Iterators: repeated, partition
using CUDA

In [ ]:
train_datai = CSV.read("csvTrainImages 13440x1024.csv", DataFrame)
test_datai = CSV.read("csvTestImages 3360x1024.csv", DataFrame)
train_labeli = CSV.read("csvTrainLabel 13440x1.csv", DataFrame)
test_labeli = CSV.read("csvTestLabel 3360x1.csv", DataFrame)

In [ ]:
train_data = reshape(Array(train_datai),:,32,32)/255;
test_data = reshape(Array(test_datai),:,32,32)/255;
train_data = reverse(train_data,dims=(2));
test_data = reverse(test_data,dims=(2));

train_label = reshape(onehotbatch(Array(train_labeli),1:28),28,13439);
test_label = reshape(onehotbatch(Array(test_labeli),1:28),28,3359);

In [ ]:
X = permutedims(train_data,[2,3,1])
X = reshape(X,32,32,1,13439)

In [ ]:
function mini_batches_train(x,y)
    ret_arr = []
    for i = 0:88
        X_chosen = reshape(x[:,:,1,i*151+1:i*151+151],32,32,1,151)
        Y_chosen = Array{Float64}(y[:,i*151+1:i*151+151])
        
        ret_arr = push!(ret_arr,(X_chosen,Y_chosen))
    end
    return ret_arr
end
train = (mini_batches_train(X,train_label))

In [ ]:
function test_batch(x,y)
    return [x,y]
end
test = test_batch(test_data,test_label)

In [ ]:
model = Chain(
    
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    x -> maxpool(x, (2,2)),

    
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    x -> maxpool(x, (2,2)),

    
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    x -> maxpool(x, (2,2)), # 4x4

    
    x -> reshape(x, :, size(x, 4)),
    Dense(512, 28),

    
    softmax,
)


In [ ]:
train = gpu(train)
model = gpu(model)